In [1]:
# Import dependencies
import pandas as pd
import plotly.graph_objects as go
import datetime as dt

## Data Processing

In [2]:
# Read in team pitching logs
pitching_df = pd.read_csv('Resources/Data/pitching_logs.csv')
pitching_df.head()

,Rk,Gtm,Date,IP,H,R,ER,UER,BB,SO,HR,HBP,ERA
0,1,1,Apr 8,11.0,9,5,4,1,3,11,1,1,3.27
1,2,2,Apr 9,9.0,5,2,2,0,4,12,1,0,2.70
2,3,3,Apr 10,9.0,5,4,4,0,4,8,1,1,3.10
3,4,4,Apr 11,9.0,10,3,3,0,0,10,1,1,3.08
4,5,5,Apr 12,9.0,5,0,0,0,0,10,0,0,2.49


In [3]:
# Create SO/9 and HR/9 columns
pitching_df['totalSO'] = pitching_df['SO'].cumsum()
pitching_df['totalHR'] = pitching_df['HR'].cumsum()
pitching_df['totalWH'] = pitching_df['BB'].cumsum() + pitching_df['H'].cumsum()
pitching_df['floatIP'] = pitching_df['IP'].apply(lambda x: int(x) + (x % 1)*10/3)
pitching_df['totalIP'] = pitching_df['floatIP'].cumsum()
pitching_df['SO/IP'] = pitching_df['totalSO'] / pitching_df['totalIP']
pitching_df['HR/9'] = pitching_df['totalHR'] / pitching_df['totalIP'] * 9
pitching_df['WHIP'] = pitching_df['totalWH'] / pitching_df['totalIP']
pitching_df = pitching_df.round({'floatIP': 2, 'totalIP': 2, 'SO/9': 2, 'HR/9': 2, 'WHIP': 2})
pitching_df.tail()

,Rk,Gtm,Date,IP,H,R,ER,UER,BB,SO,...,HBP,ERA,totalSO,totalHR,totalWH,floatIP,totalIP,SO/IP,HR/9,WHIP
103,104,104,Aug 1,9.0,7,2,2,0,3,3,...,0,3.18,955,100,1029,9.0,934.0,1.022484,0.96,1.10
104,105,105,Aug 2,9.0,8,8,7,1,4,8,...,0,3.22,963,103,1041,9.0,943.0,1.021209,0.98,1.10
105,106,106,Aug 3,9.0,10,7,7,0,1,9,...,0,3.25,972,107,1052,9.0,952.0,1.021008,1.01,1.11
106,107,107,Aug 5,8.0,5,4,4,0,6,9,...,0,3.26,981,107,1063,8.0,960.0,1.021875,1.00,1.11
107,108,108,Aug 6,8.0,5,1,1,0,2,5,...,0,3.24,986,107,1070,8.0,968.0,1.018595,0.99,1.11


In [4]:
# Filter for necessary columns
pitching_df = pitching_df[['Rk', 'Gtm', 'Date', 'ERA', 'SO/IP', 'HR/9', 'WHIP']]
pitching_df.head(15)

,Rk,Gtm,Date,ERA,SO/IP,HR/9,WHIP
0,1,1,Apr 8,3.27,1.000000,0.82,1.09
1,2,2,Apr 9,2.70,1.150000,0.90,1.05
2,3,3,Apr 10,3.10,1.068966,0.93,1.03
3,4,4,Apr 11,3.08,1.078947,0.95,1.05
4,5,5,Apr 12,2.49,1.085106,0.77,0.96
5,6,6,Apr 13,2.89,1.035714,1.12,0.98
6,7,7,Apr 14,2.49,1.061538,0.97,1.00
7,8,8,Apr 15,2.26,1.030837,0.83,1.07
8,9,9,Apr 16,2.23,1.003937,0.85,1.07
9,10,10,Apr 17,2.53,1.089928,0.78,1.12


In [5]:
# Convert Date string to datetime
pitching_df['Date'] = pitching_df['Date'] + ' 2022'
pitching_df['Date'] = pd.to_datetime(pitching_df['Date'], format="%b %d %Y")
pitching_df['Day'] = pitching_df['Date'].apply(lambda x: x.strftime('%b %d'))
pitching_df.dtypes

Rk                int64
Gtm               int64
Date     datetime64[ns]
ERA             float64
SO/IP           float64
HR/9            float64
WHIP            float64
Day              object
dtype: object

## Viz

In [29]:
# Create variables for use in chart
x = pitching_df['Date']
y = pitching_df['ERA']
#hover = pitching_df['Day']
#y2 = pitching_df['WHIP']
#y3 = pitching_df['SO/IP']
#y4 = pitching_df['HR/9']

navy = '#0C2340'
gray = '#C4CED3'


# Create traces
fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=y, mode='lines', line_color='white'))#, customdata=hover))# name='ERA',  title="layout.hovermode='x unified'"))
# fig.add_trace(go.Scatter(x=x, y=y2, name='WHIP', mode='lines+markers', line_color='#d4db6b'))
# fig.add_trace(go.Scatter(x=x, y=y3, name='SO/IP', mode='lines+markers', line_color='#4fbfdb'))
# fig.add_trace(go.Scatter(x=x, y=y4, name='HR/9', mode='lines+markers', line_color='#4fbfdb'))


# Add traces for annotations and text for end of lines
for i, d in enumerate(fig.data):
    fig.add_trace(go.Scatter(x=[d.x[-1]],
                             y=[d.y[-1]],
                             mode='markers+text',
                             text=['<b>' + str(y) + '</b>' for y in [d.y[-1]]],
                             textfont = dict(
                                 size=18,
                                 color='white'),
                             textposition='middle right',
                             marker = dict(
                                 size=22,
                                 color = d.line.color,
                                 line=dict(
                                     width=2.5,
                                     color=navy)),
                             showlegend=False))


# Format traces
fig.update_traces(
#     marker=dict(
#         size=12,
#         color=gray),
#         line=dict(
#             width=0.5,
#             color=navy)),
    line=dict(
        shape='spline',
        smoothing=1.3,
        width=15),
    #hover_name=None,
    #hovertemplate='<br>  <b>%{customdata}</b><br>ERA: <b>%{y}</b><br>',
    hovertemplate='<br><br>ERA: <b>%{y}</b><br>',
    xhoverformat='<b>%b %d</b>'
)


# Format axes
fig.update_xaxes(
#     tickmode='array',
#     ticktext=['<b>' + date + '</b>' for date in ['April','May','Jun','Jul','Aug']],
#     tickvals=['2022-04-15','2022-05-15','2022-06-15','2022-07-15','2022-08-15'],
    dtick='M1',
    tickformat='<b>%b</b>',
    ticklabelmode='period',
#     dtick=2*7*24*60*60*1000,
#     tick0="2022-04-01"
#     ticks= "outside",
#     tickcolor= gray,
#     ticklen=8, 
#     minor=dict(
#         ticklen=4,  
#         dtick=2*7*24*60*60*1000,
#         tick0="2022-05-22"), 
    tickfont=dict(
        size=24,
        color=gray),
    gridcolor='rgb(20,45,75)',
    gridwidth=2
    #showgrid=False
)

fig.update_yaxes(
    title=dict(
        text="<br><b>ERA</b>",
        font=dict(
            size=26,
            color='white')),
    tickfont=dict(
        size=20,
        color=gray),
    tickmode='array',
    ticktext=['<b>' + x + '</b>' for x in ['2.00','2.25','2.50','2.75','3.00','3.25','3.50']],
    tickvals=[2, 2.25, 2.5, 2.75, 3, 3.25, 3.5],
    zerolinecolor='rgb(20,45,75)',
    zerolinewidth=2,
    gridcolor='rgb(20,45,75)',
    gridwidth=2,
)


# Format layout
fig.update_layout(
    title=dict(
        text="<b>Yankees Team ERA</b><br>Time to Blame Blake?",
        y=0.9,
        x=0.5,
        xanchor='center',
        yanchor='top',
        font=dict(
            size=30,
            color='white')),
    xaxis_range=['2022-04-01','2022-08-17'],
    yaxis_range=[1.9,3.6],
    showlegend=False,
    hovermode='x',
    hoverlabel=dict(
        bgcolor=navy,
        font_size=16),
        #font_family="Rockwell"
#     legend=dict(
#         title_text='',
#         orientation='h',
#         yanchor="top",
#         y=0.947,
#         xanchor="left",
#         x=0.345,
#         font=dict(
#             size=18,
#             color='white'),
#         bordercolor=gray,
#         borderwidth=2,
#         bgcolor=navy),
    paper_bgcolor=navy,
    plot_bgcolor=navy,
    #width=1500,
    margin=dict(t=110, b=70, pad=3)
)

In [13]:
# Create variables for use in chart
x = pitching_df['Date']
#y1 = pitching_df['ERA']
y2 = pitching_df['WHIP']
#y3 = pitching_df['SO/IP']
y4 = pitching_df['HR/9']

navy = '#0C2340'
gray = '#C4CED3'


# Create traces
fig = go.Figure()

#fig.add_trace(go.Scatter(x=x, y=y1, name='ERA', mode='lines+markers', line_color='#db5f4f'))
fig.add_trace(go.Scatter(x=x, y=y2, name='WHIP', mode='lines', line_color='#d4db6b'))
#fig.add_trace(go.Scatter(x=x, y=y3, name='SO/IP', mode='lines', line_color='#4fbfdb'))
fig.add_trace(go.Scatter(x=x, y=y4, name='HR/9', mode='lines', line_color='#4fbfdb'))


# Add traces for annotations and text for end of lines
for i, d in enumerate(fig.data):
    fig.add_trace(go.Scatter(x=[d.x[-1]],
                             y=[d.y[-1]],
                             mode='markers+text',
                             text=['<b>' + str(y) + '</b>' for y in [d.y[-1]]],
                             textfont = dict(
                                 size=18,
                                 color='white'),
                             textposition='middle right',
                             marker = dict(
                                 color = d.line.color,
                                 line=dict(
                                     width=d.marker.line.width,
                                     color=d.marker.line.color)),
                             showlegend=False))


# Format traces
fig.update_traces(
#     marker=dict(
#         size=10,
#         line=dict(
#             width=0.5,
#             color='white')),
    line=dict(
        shape='spline',
        smoothing=1.3,
        width=12)
)


# Format axes
fig.update_xaxes(
    tickmode='array',
    ticktext=['<b>' + date + '</b>' for date in ['April','May','Jun','Jul','Aug']],
    tickvals=['2022-04-15','2022-05-15','2022-06-15','2022-07-15','2022-08-15'],
#     ticks= "outside",
#     tickcolor= gray,
#     ticklen=8, 
#     minor=dict(
#         ticklen=4,  
#         dtick=2*7*24*60*60*1000,
#         tick0="2022-05-22"), 
    tickfont=dict(
        size=18,
        color=gray),
    showgrid=False
)

fig.update_yaxes(
    tickfont=dict(
        size=18,
        color=gray),
    tickmode='array',
    ticktext=['<b>' + x + '</b>' for x in ['0.00','0.25','0.50','0.75','1.00','1.25']],
    tickvals=[0, 0.25, 0.5, 0.75, 1, 1.25],
    zerolinecolor='rgb(20,45,75)',
    zerolinewidth=2,
    gridcolor='rgb(20,45,75)',
    gridwidth=2,
)


# Format layout
fig.update_layout(
    title=dict(
        text="<b>Clay Holmes</b><br>Recent Decline in Performance",
        y=0.9,
        x=0.5,
        xanchor='center',
        yanchor='top',
        font=dict(
            size=28,
            color='white')),
#     xaxis_range=['2022-05-10','2022-08-13'],
    yaxis_range=[0.5,1.25],
    legend=dict(
        title_text='',
        orientation='h',
        yanchor="top",
        y=0.947,
        xanchor="left",
        x=0.345,
        font=dict(
            size=18,
            color='white'),
        bordercolor=gray,
        borderwidth=2,
        bgcolor=navy),
    paper_bgcolor=navy,
    plot_bgcolor=navy,
    margin=dict(b=70, pad=3)
)